<a href="https://colab.research.google.com/github/harrisb002/CS_480/blob/main/Projects/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Introduction: Sorting by Reversal

---
## Overview
In this project, I will implement the search algorithms—Breadth-First Search (BFS),  Iterative Deepening Search (IDS), and A* search algorithm - to solve the *sorting by reversal* problem. This problem has practical applications in computational molecular biology, specifically in assessing gene sequence similarities, as described in the book *Computational Molecular Biology: An Algorithmic Approach* by Pavel A. Pevzner.

---
### Problem Statement
The starting state of the problem is an arbitrary permutation, π, of numbers `[1, 2, …, n]`, while the goal state is the sorted sequence `[1, 2, …, n]`. To transition from the starting state to the goal state, one must select a contiguous block within the permutation and reverse it. The objective is to sort the permutation in the fewest number of reversals.

**Example:**
Given the permutation `[1, 3, 4, 2, 6, 5]`, reversing the block `[4, 2, 6]` results in the permutation `[1, 3, 6, 2, 4, 5]`.

---
### Project Requirements
The following algorithms will be implemented:
1. **Breadth-First Search (BFS)**
2. **Iterative Deepening Search (IDS)**
3. **A* Search** using an admissible heuristic `h` (described below).

These implementations will use a tree-search approach, avoiding only trivial cycles of length 2. This means that for each node, all successors, except the parent node, are added to the fringe, regardless of whether they are already in or have been removed from the fringe.

---
### Heuristic Function for A* Search
In the A* search implementation, we use a heuristic function `h` defined as follows:
- For a given permutation π = `[x1, x2, …, xn]`, a break-point `j` is an integer such that `1 ≤ j ≤ n − 1` and `xj` and `xj+1` are not consecutive integers.
- **Example:** The permutation `[5, 4, 6, 1, 2, 7, 3]` has four break-points.
- The heuristic function `h(π) = 1/2 ×` the number of break-points.
- A binary heap is used to implement the fringe (Open set) for A* search.

---
### Program Execution
When the program is executed, it will prompt the user to enter a permutation `P` (e.g., `3 4 5 1 8 6 2 7`). The output will include:
- The sequence of states, from the initial permutation to the sorted sequence, found by BFS and A* search, with one state per line.
- The total CPU time used by each algorithm.
- The total number of nodes added to the queue.
- The maximum size of the queue during the search process.

The tree-search version eliminates the need to maintain both OPEN and CLOSED sets.

---
### Constraints and Testing
The program will be tested on input sizes up to `n ≤ 10`.
---

# Imports and Input

In [14]:
import time
from collections import deque
from heapq import heappush, heappop

# Helper Functions

In [15]:
def get_input():
  """
  Prompt for input as ints
  Returns the input as a list of ints.
  """
  input_string = input("Enter the numbers separated by spaces: ")
  input_list = input_string.split() # Split by spaces
  return [int(num) for num in input_list] # Convert to ints

def is_sorted(permutation):
  """
  Check if perm is sorted in asc order.
  Parameters:
      permutation (list): List of ints representing the curr perm.
  Returns:
      bool: True if list is sorted in asc order, else False.
  """
  return list(permutation) == sorted(permutation)

def reverse_block(state, start, end):
  """
  Reverse a block of the list between specified start and end indices.
  Parameters:
    state (list): List of ints representing the curr state of the perm.
    start (int): Start index of the block to reverse.
    end (int): End index of the block to reverse.
  Returns:
    list: A new list with the specified block reversed.
  """
  new_state = list(state)  # Convert tuple to list for mutability
  new_state[start:end + 1] = reversed(state[start:end + 1])
  return tuple(new_state)  # Convert back to tuple

def calculate_breakpoints(permutation):
  """
  Breakpoint is a position where two adjacent numbers are not consecutive.
  Parameters:
    permutation (list): List of ints representing the curr perm
  Returns:
    int: num of breakpoints in the perm
  """
  breakpoints = 0
  for i in range(len(permutation) - 1):
      if abs(permutation[i] - permutation[i + 1]) != 1:
          breakpoints += 1
  return breakpoints

# Breadth-First Search (BFS) Implementation

In [16]:
def bfs_sort_reversal(start_state):
  """
  Find min num of reversals needed to sort the perm.

  Parameters:
    start_state (list): Initial unsorted perm.

  Returns:
    dict: A dict containing the results of the search, including:
      - 'moves' (int): Num of moves to sort the perm.
      - 'path' (list): List of states from start to goal state.
      - 'states_visited' (int): Total num of states visited.
      - 'max_queue_size' (int): Max size of the queue encountered.
      - 'cpu_time' (float): Total CPU time taken.
  """
  # Goal state as a sorted tuple of the input perm
  goal_state = tuple(sorted(start_state))

  # Init the queue with the start state and path
  # Queue elements are tuples containing (current state, path to current state)
  queue = deque([(tuple(start_state), [start_state])])

  # Track visited states to prevent reprocessing
  visited = {tuple(start_state)}

  # Init metrics for tracking
  start_time = time.time()
  nodes_visited = 0
  max_queue_size = 0

  # BFS loop
  while queue:
    # Update max queue size if the curr size exceeds the prev max
    max_queue_size = max(max_queue_size, len(queue))

    # Dequeue the first element, updating the curr state and its path
    current_state, path = queue.popleft()
    nodes_visited += 1

    # goal state found, return the results
    if current_state == goal_state:
      end_time = time.time()
      return {
        'moves': len(path) - 1,
        'path': path,
        'states_visited': nodes_visited,
        'max_queue_size': max_queue_size,
        'cpu_time': end_time - start_time
      }

    # Gen successors by reversing all possible sublists within the curr state
    n = len(current_state)
    for i in range(n):
      for j in range(i + 1, n):
        # Gen a new state by reversing the segment between indices i and j
        new_state = tuple(reverse_block(current_state, i, j))

        # If new state hasn't been visited, enqueue it
        if new_state not in visited:
          visited.add(new_state)
          queue.append((new_state, path + [new_state]))

  # If no solution, return fail message / metrics
  end_time = time.time()
  return {
    'moves': None,
    'path': [],
    'states_visited': nodes_visited,
    'max_queue_size': max_queue_size,
    'cpu_time': end_time - start_time
  }

# Iterative Deepening Search (IDS) Implementation


In [12]:
def ids(permutation):
  """
  Perform Iterative Deepening Search (IDS) to find min numb of reversals
  needed to sort perm in asc order.
  Args:
    permutation (list): A list of ints representing the given perm.
  Returns:
    dict: A dict containing the results of the search, including:
      - 'moves' (int): Num of moves to sort the perm.
      - 'path' (list): List of states from start to goal state.
      - 'states_visited' (int): Total num of states visited.
      - 'max_queue_size' (int): Max size of the queue encountered.
      - 'cpu_time' (float): Total CPU time taken.
  """

  def dls(permutation, depth, path, visited, total_states):
    """
    Perform Depth-Limited Search (DLS) with the specified depth limit.
    Args:
      permutation (list): The current state of the perm.
      depth (int): The current depth limit.
      path (list): List of perms representing the path from the initial state.
      visited (set): Set of visited perms to avoid revisiting.

    Returns:
      tuple: A tuple containing the path to the sorted perm and a boolean
        indicating if a solution was found.
    """

    # Base case: if depth limit is 0 and perm is sorted, solution is found.
    if depth == 0 and is_sorted(permutation):
      return path, True, total_states

    # If we still have depth to explore, continue with recursive search.
    elif depth > 0:
      visited.add(tuple(permutation))  # Mark the curr perm as visited.

      # Iterate over all possible reversal pairs (i, j).
      for i in range(len(permutation)):
        for j in range(i + 1, len(permutation)):
          # Reverse the block between indices i and j.
          new_permutation = reverse_block(permutation, i, j)
          total_states += 1  # Increment total_states counter

          # Continue with the search only if the new perm is not visited.
          if new_permutation not in visited:
            # Rec call with decreased depth and updated path.
            result_path, found, total_states = dls(new_permutation, depth - 1, path + [new_permutation], visited, total_states)

            # If a solution is found at this depth, return path.
            if found:
              return result_path, True, total_states


      # Backtrack: remove the curr perm from visited set to allow others.
      visited.remove(tuple(permutation))

    # Return empty path and False if no solution was found with curr depth.
    return [], False, total_states

  # Init depth level for IDS and tracking variables.
  depth = 0
  total_states = 0
  start_time = time.time()

  while True:
    visited = set()  # Reset the visited set for each new depth level.

    # Run dls at the current depth level.
    path, found, total_states = dls(permutation, depth, [permutation], visited, total_states)

    # Accumulate the numof visited states across all depths.
    total_states += len(visited)

    # If DLS found a solution, print the results and exit.
    if found:
      end_time = time.time()
      cpu_time = end_time - start_time

      # Return a dict of results
      return {
      "path": path,
      "cpu_time": cpu_time,
      "states_visited": total_states,
      "moves": len(path) - 1,  # Sub 1 because init state is part the path
      "max_queue_size": None  # N/a
    }

    # Inc the depth limit for the next iteration of IDS.
    depth += 1

# A* Implementation


In [17]:

def a_star_search(start_permutation):
  """
  Find the min num of reversals to sort a perm
  Parameters:
  - start_permutation: list of ints representing the initial unsorted permutation
  Returns:
    dict: A dict containing the results of the search, including:
      - 'moves' (int): Num of moves to sort the perm.
      - 'path' (list): List of states from start to goal state.
      - 'states_visited' (int): Total num of states visited.
      - 'max_queue_size' (int): Max size of the queue encountered.
      - 'cpu_time' (float): Total CPU time taken.
  """
  start_time = time.time()
  visited = set()  # Set to track visited states
  queue = []       # Priority queue for managing search frontier
  initial_heuristic = calculate_breakpoints(start_permutation) / 2

  # (heuristic, cost, state, path)
  heappush(queue, (initial_heuristic, 0, start_permutation, []))

  # Init tracking variables
  max_queue_size = 0     # Tracks the max queue size during the search
  nodes_expanded = 0     # Counter for the total num of expanded nodes
  goal_state = sorted(start_permutation)  # Define goal state as sorted perm

  # A* Search loop
  while queue:
    # Update the max queue size if the curr size exceeds previous max
    max_queue_size = max(max_queue_size, len(queue))

    # Pop the node with the lowest heuristic value
    heuristic, cost, current_permutation, path = heappop(queue)

    # Check if the curr perm is the goal state
    if current_permutation == goal_state:
      total_time = time.time() - start_time
      return {
        'moves': cost,
        'path': path + [current_permutation],
        'states_visited': nodes_expanded,
        'max_queue_size': max_queue_size,
        'cpu_time': total_time
      }

    # Mark the current permutation as visited
    visited.add(tuple(current_permutation))

    # Generate successors by reversing all possible subarrays
    for i in range(len(current_permutation)):
      for j in range(i + 2, len(current_permutation) + 1):
        # Create new perm by reversing the segment between indices i and j
        # Use reverse_block to reverse the specified section.
        new_permutation = list(reverse_block(current_permutation, i, j - 1))#inclusive

        # If new state hasn't been visited, calculate heuristic and add to queue
        if tuple(new_permutation) not in visited:
          # Heuristic is half the num of breakpoints plus the current cost + 1
          new_heuristic = (calculate_breakpoints(new_permutation) / 2) + cost + 1
          heappush(queue, (new_heuristic, cost + 1, new_permutation, path + [current_permutation]))
          nodes_expanded += 1

# Printing output:


In [8]:
def print_output(result, algorithm_name):
  """
  Parameters:
    result (dict): A dict with the results of the search, with the following keys:
      - 'moves' (int): num of moves to sort the permutation.
      - 'path' (list): List of states from start to goal state.
      - 'states_visited' (int): Total num of states visited during the search.
      - 'max_queue_size' (int): Max size of the queue (for BFS and A*).
      - 'cpu_time' (float): Total CPU time taken by the algo.
    algo_name (str): Name of the search algo (e.g., "BFS", "IDS", "A*").
  """
  print(f"Output of {algorithm_name}:")
  print(f"Number of Moves to Sort: {result['moves']}")
  for state in result['path']:
    print(state)
  print(f"The total num of states visited was {result['states_visited']}")

  # Conditionally print max queue size if provided in the result
  if 'max_queue_size' in result:
    print(f"The max size of the queue was {result['max_queue_size']}")

  print(f"The total CPU time was {result['cpu_time']:.5f} seconds\n")

# Main

In [18]:
if __name__ == "__main__":

  # Get input from user
  start_permutation = get_input()

  # Run BFS
  bfs_result = bfs_sort_reversal(start_permutation)
  print_output(bfs_result, "BFS")

  # Run A*
  astar_result = a_star_search(start_permutation)
  print_output(astar_result, "A*")

  # Run IDS
  ids_result = ids(start_permutation)
  print_output(ids_result, "IDS")

Enter the numbers separated by spaces: 1 3 5 7 9 2 4 6 8 10
Output of BFS:
Number of Moves to Sort: 5
[1, 3, 5, 7, 9, 2, 4, 6, 8, 10]
(1, 2, 9, 7, 5, 3, 4, 6, 8, 10)
(1, 2, 6, 4, 3, 5, 7, 9, 8, 10)
(1, 2, 3, 4, 6, 5, 7, 9, 8, 10)
(1, 2, 3, 4, 5, 6, 7, 9, 8, 10)
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
The total num of states visited was 1096477
The max size of the queue was 1789929
The total CPU time was 87.50556 seconds

Output of A*:
Number of Moves to Sort: 5
[1, 3, 5, 7, 9, 2, 4, 6, 8, 10]
[1, 2, 9, 7, 5, 3, 4, 6, 8, 10]
[1, 2, 6, 4, 3, 5, 7, 9, 8, 10]
[1, 2, 3, 4, 6, 5, 7, 9, 8, 10]
[1, 2, 3, 4, 5, 6, 7, 9, 8, 10]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
The total num of states visited was 5569
The max size of the queue was 5442
The total CPU time was 0.02731 seconds

Output of IDS:
Number of Moves to Sort: 5
[1, 3, 5, 7, 9, 2, 4, 6, 8, 10]
(1, 2, 9, 7, 5, 3, 4, 6, 8, 10)
(1, 2, 6, 4, 3, 5, 7, 9, 8, 10)
(1, 2, 3, 4, 6, 5, 7, 9, 8, 10)
(1, 2, 3, 4, 5, 6, 7, 9, 8, 10)
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)


In [13]:
# Get input from user
start_permutation = get_input()

# Run IDS
ids_result = ids(start_permutation)
print_output(ids_result, "IDS")

Enter the numbers separated by spaces: 9 1 6 3 4 5 7 2 8
Output of IDS:
Number of Moves to Sort: 5
[9, 1, 6, 3, 4, 5, 7, 2, 8]
(1, 9, 6, 3, 4, 5, 7, 2, 8)
(1, 2, 7, 5, 4, 3, 6, 9, 8)
(1, 2, 3, 4, 5, 7, 6, 9, 8)
(1, 2, 3, 4, 5, 6, 7, 9, 8)
(1, 2, 3, 4, 5, 6, 7, 8, 9)
The total num of states visited was 2248629
The max size of the queue was None
The total CPU time was 4.41656 seconds

